## Lab 7: Heterogenous treatment effects, Experiments

In [133]:
import numpy as np
from datascience import Table
%matplotlib inline

## Part 1. Donor Influence/Corruption, Continued

Let's continue to explore these idea of how money may influence political decisions in a slightly different way.

Imagine a company called InfluenceCo, who needs local governments to make favorable decisions for them to be able to run their business profitably. Perhaps they are a ride-sharing company who needs cities to allow them to operate.

We will consider the decisions made by mayors of a large number of cities where InfluenceCo wants to do business. As before, we will assume that mayors vary in their "pro-business" ideology, but also may make decisions based on political donations (or outright bribes).

InfluenceCo wants mayors to support a policy that would help their business. To simplify, let's assume the policy in question is entirely at the discretion of the mayor.

First suppose that the mayors are all dedicated public servants who will only support the policy if they think it is good for their city. Let's given them a pro-business ideology that ranges from 0 to 1, and put it in a table.

In [5]:
n_mayors = 2000
mayor_ideol = np.random.rand(n_mayors) 
mayor_dataA = Table().with_column("Ideology", mayor_ideol)

A natural assumption is that those with a higher ideology are more likely to support it. Here we will capture this by assuming that the utility for supporting the policy is:
$$
u_{support} = \text{ideology} - e
$$
where $e$ is a random number between 0 and 1 (use `np.random.rand` for this). The utility to not supporting is zero.

**Question 1.1. Write code to create a variables which correspond to the mayor utility for supporting the policy, and a variable equal to 1 if they choose to support it and 0 otherwise. Add the binary support variable to the `mayor_dataA` table with the name "Support".**

In [1]:
#Code for 1.1

Last week we considered a causal process where donations and politician behavior were confounded by their ideology. Now let's think of a process of reverse causation, where politician behavior causes donations.

Here is a data generating process which captures the idea that politicians are more likely to receive donations when they support policies that donors like. In the code below we can think of .8 as the "cost" to donating.

In [109]:
u_donate = .5*mayor_dataA.column("Support") + np.random.rand(n_mayors)
mayor_dataA = mayor_dataA.with_column("Donate", 1*(u_donate > .8))
mayor_dataA

Ideology,Support,Donate
0.348383,0,0
0.435622,0,0
0.833804,1,1
0.39128,1,0
0.756738,1,0
0.801768,1,0
0.305731,1,1
0.891246,1,1
0.337161,0,1
0.626508,0,0


**Question 1.2. Give a short theoretical explanation for why politican support decisions might cause donations which is consistent with this code.**

*Words for 1.2*

**Question 1.3. Compute the difference in the proportion of mayors who support the policy who receieved donations vs. not. Given this data generating process, does this difference reflect a causal effect of donations on voting behavior or selection bias (or both)? That is, do mayors who vote differently do so because of the donations, or are they just different for other reasons?**

In [2]:
#Code for 1.3

*Words for 1.3*

Throughout this lab we will be taking many of differences of means, and the table notation for this is a bit repetitive. So let's make a function that computes differences of means more efficiently. This will work for any "treatment" variable that takes on values 0 and 1.

In [19]:
def diffmean(outcome, treatment, data):
    mean1 = np.mean(data.where(treatment, 1).column(outcome))
    mean0 = np.mean(data.where(treatment, 0).column(outcome))
    return(mean1-mean0)

**Question 1.4. Compute the same difference of means from part 1.3 using the `diffmean` function. (Recall that for a variable that takes on values of 0 or 1, the mean gives us the proportion of 1s. So in this case, the `diffmean` function will return the difference in proportions.)**

In [4]:
# Code for 1.4

Something missing in this process is the idea that mayors probably want to receive donations, and so if they anticipate receiving donations if they support a policy that could influence their decisions. (If your answer to 1.3 assumed the previous data generating process captured that you might want to change it!)

Here is one way to capture this idea. Let's create a variable called `b_don=.3` which reflects how much mayors like getting donations. The mayor utility for supporting the policy without a donation is the same as above, but let's now call this `u_support0`. Their utility for supporting the policy if they do get a donation is the utility with no donation plus `b_don`. The utility to not supporting is 0. Here is code for this:

In [111]:
b_don = .3
u_support0 = mayor_ideol - np.random.rand(n_mayors)
u_support1 = u_support0 + b_don
u_nosupport = 0

We will store the data from this process in a table called `mayor_dataB`. Let's start by making a copy of the previous mayors table but without the support and donate variables, which will be different in this version.

In [112]:
mayor_dataB = mayor_dataA.drop(["Support", "Donate"])

**Question 1.5. Write code to create two variables and add them to `mayor_dataB`, the first of which indicate if the mayor prefers to support the policy if they don't get a donation (call this "Support0") and the second which indicates if the mayor prefers to support the policy if they do get a donation (call this "Support1").** 

In [5]:
# Code for 1.5

Now let's suppose the donors can anticipate who will support the policy with a donation, and want to reward politicians who do so (but also consider other factors too). The following code captures this idea (it assumes you named the variable correctly in the previous block!) Again, we are implicitly  assuming the "cost" to donating is .8.

In [114]:
u_donate2 = .5*mayor_dataB.column("Support1") + np.random.rand(n_mayors)
mayor_dataB = mayor_dataB.with_column("Donate", 1*(u_donate2 > .8))
mayor_dataB

Ideology,Support0,Support1,Donate
0.348383,1,1,0
0.435622,1,1,0
0.833804,0,1,1
0.39128,0,0,0
0.756738,1,1,1
0.801768,1,1,0
0.305731,0,1,1
0.891246,1,1,1
0.337161,0,0,0
0.626508,1,1,1


**Question 1.6. Use the `np.where` function to create a variable which indicates the realized support decision (i.e., "Support0" with no donation and "Support1" with a donation), and add it to the table with the name "Support"**

In [6]:
# Code for 1.6

 What is the causal effect of the donation here? It is the difference between the support choice with a donation versus not, which will not necessarily be the same for each mayor. 
 
 **Question 1.7.Write code to create a variable and add it to `mayor_dataB` with the name "Causal".**

In [7]:
# Code for 1.7

**Question 1.8. What is the Average Treatment Effect of a donation here? It should be a number between 0 and 1. What does the Average Treatment Effect mean in words?**

In [8]:
# Code for 1.8

*Words for 1.8*

**Question 1.9 what is the Average Treatment Effect on the Treated? What does this mean in words? (Hint: it's a bit different than your answer to 1.8)**

In [9]:
# Code for 1.9

*Words for 1.9*

**Question 1.10 Write code to confirm that the Difference of Means is equal to the ATET plus selection bias. Remember you can use the `diffmean` function for both the difference of means and selection bias!**

In [10]:
#Code for 1.10

## Part 2. A Fake Experiment

While it would probably be infeasible and unethical to run an experiment where we get donors to randomize which politicians they give money to, we can see how this would play out hypothetically with our simulated data. 

Let's again use the potential outcomes from the previous simulated data, putting it in a table called mayor_dataR (for "randomized")

In [121]:
mayor_dataR = mayor_dataB.select(["Ideology", "Support0", "Support1", "Causal"])
mayor_dataR

Ideology,Support0,Support1,Causal
0.348383,1,1,0
0.435622,1,1,0
0.833804,0,1,1
0.39128,0,0,0
0.756738,1,1,0
0.801768,1,1,0
0.305731,0,1,1
0.891246,1,1,0
0.337161,0,0,0
0.626508,1,1,0


A simple way to randomly decide who gets a donation is to use the `np.random.binomial` function that we used in lab 3. This function takes three arguments. For example the following can be thought of as flipping 10 "biased" coins that are heads with probability .4, and reporting 1 if heads and 0 if tails.

In [122]:
np.random.binomial(1, .4, 10)

array([1, 1, 0, 0, 0, 1, 0, 1, 0, 0])

If we change the first argument, that changes "how many coins" we flip for each simulation, e.g., the following simulates flippling 3 biased coins and counting how many heads there are, and doing this 10 times:

In [123]:
np.random.binomial(3, .4, 10)

array([2, 0, 1, 1, 2, 1, 2, 2, 1, 0])

If we increase the second argument, we will tend to get more "heads":

In [124]:
np.random.binomial(3, .8, 10)

array([2, 3, 3, 3, 2, 1, 3, 3, 2, 3])

And if we increase the third argument, we do more sets of flips.

In [125]:
np.random.binomial(3, .8, 20)

array([3, 2, 2, 2, 3, 2, 3, 2, 3, 3, 3, 1, 2, 3, 3, 2, 3, 1, 2, 3])

**Question 2.1. Write code which randomly says whether each mayor gets a donation or not (think of it as 1 coin flip), where each gets a donation with probability .5. Add this to the `mayor_dataR` table with the name "DonateR". (Hint: recall the number of mayors is `n_mayors`.**

In [11]:
# Code for 2.1

**Question 2.2 Make a variable for the realized support choice with the random donation, and add it to `mayor_dataR` with the name "Support".**

In [12]:
# Code for 2.2

**Question 2.3 Compute the difference in mean of "Support" among those received a (random) donation versus not, and compare this to the Average Treatment Effect and the Average Treatment Effect on the Treated (hint: use the "Causal" variable).**

In [13]:
# Code for 2.3

*Words for 2.3*

## Part 3. A real Experiment

Now let's see how these ideas play out with some real data. Unlike with the simulations, we can't check whether there isn't any selection bias driven by differences in treatment and control groups, since in "real world" mode we don't get to know the potential outcomes. But with the confidence gained by our simulations, if the treatment is randomized, then we can assume that selection bias is likely to be small. We can also check that the treatment and control groups are otherwise similar.

The example we will use is from <a href="https://www.jstor.org/stable/26379536?seq=1#metadata_info_tab_contents">this</a> paper, which aimed to see if sending letters to political elites can encourage them to get women more involved in leadership positions.

Quoting directly from the article: "We ran a field experiment with the cooperation of a state Republican Party in a state with low levels of women's representation (Utah). Party leaders were concerned that although women comprised about half of the party activists who attended neighborhood caucus meetings, women typically accounted for only 20-25% of the delegates elected from these meetings to attend the state nominating convention. We randomly assigned over 2,000 precinct chairs to receive one of four letters from the state party chair prior to these neighborhood caucus meetings. The treatments were a neutral placebo control (Control), a request to recruit 2-3 women to run as state delegates (Supply), a request to read a letter at the precinct meeting encouraging attendees to elect more women as delegates (Demand), and a request to both recruit women and read the letter (Supply+Demand)."

Note that in this case, like in many real experiments, there is not just one treatment and one control. Since precinct chairs could get neither, one, or both, there are four "treatment statuses". We will see how these groups differ individually, as well as comparing "supply to no supply" and "demand to no demand".

The outcomes we will look at the the proportion of female delegates and whether there are any female delegates. Here are the variables in the data:
- unique_id: Precinct ID
- "treat": treatment variable, with four possibilities:
(1) 'control': control group
(2) 'supply': supply group; party chair instructed to recruit 2-3 women
(3) 'demand': demand group; party chair reads letter at precinct convention
(4) 'both': a fourth group getting both the supply and demand treatments; party chair instructed to read letter and to recruit 2-3 women
- "prop_sd_fem2014": Outcome: Proportion of 2014 elected state delegates from that precinct who were women
- "sd_onefem2014": 1 if at least one woman was selected; 0 otherwise
- "county": County name in Utah
- "pc_male": 1 if precinct chair is male; 0 otherwise (precinct chair is person who runs precinct meeting, would read letter if assigned to do so, etc.)

In [134]:
utah = Table.read_table("electing_women.csv")
utah

unique_id,treat,prop_sd_fem2014,sd_onefem2014,county,pc_male
27215,supply,0,0,Grand,1
27386,control,0,0,Grand,0
27496,control,1,1,Grand,1
16202,demand,1,1,Daggett,1
16241,control,0.5,1,Daggett,1
26601,control,0,0,Emery,1
27551,demand,0,0,Grand,1
67237,control,0,0,San Juan,1
69699,supply,0,0,Sevier,1
86949,supply,0,0,Wasatch,1


Let's see how many observations are in each treatment group.

In [135]:
utah.group("treat")

treat,count
both,427
control,435
demand,426
supply,446


**Question 3.1. Compute the average proportion of female delegates for the each of the four possible treatments.  Interpret the results**

In [14]:
# Code for 3.1

*Words for 3.1*

If we want to estimate the effect of the supply and demand treatments individually, we can create variables that indicate whether they received the supply treatment (setting aside the demand treatment) and vice versa.

**Question 3.2.  Create a variable and add it to the table with the name "supply" which is equal to 1 if "treat" is equal to "supply" or "both", and 0 otherwise. Then create a variable and add it to the table with the name "demand" which is equal to 1 if "treat" is equal to "demand" or "both", and 0 otherwise. (Hint: there are a few ways to do this. One is to first create a variables equal to 1 for being in each treatment group. Then to determine if one received the supply treatment you can add the variable for supply and both, and that will be equal to 1 if in either.)**

In [20]:
# Code for 3.2

**Question 3.3. Compute the difference in means for the proportion of female delegates for those who did and did not receive the demand treatment, and those who did and did not receive the supply treatment. You can use the `diffmean` function since there is now a binary treatment equal to 0 or 1.**

In [16]:
#Code for 3.3

**Question 3.4. While we can't learn the potential outcomes of what delegates would have been sent with a different treatment, we can check to see if those who received different treatments did happen to be different on other dimensions. One that might matter is whether the precinct chair was a male or female (recall this is in the "pc_male" variable). Use the ``diffmean`` fuction to check if precincts with a male chair were more or less likely to send at least one female delegate.**

In [17]:
# Code for 3.4

**Question 3.5. Now use the `diffmean` function to check whether precincts who recieved the "supply" teatment were more likely to have a male precinct chair, then do the same for the "demand" treatment.**

In [18]:
# Code for 3.5

**Question 3.6. If a reader of  this study said "It's hard to say whether encouraging women to run is helpful, because male political leaders are more likely to do so and they also do other things which make women less likely to run", how could the authors respond? (Hint: think about the power of randomization in general, and what you found in the previous questions.)**

*Words for 3.6*